In [6]:
import os
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
os.environ['HF_TOKEN'] = user_secrets.get_secret("HF_TOKEN")
os.environ['OPENAI_API_KEY'] = user_secrets.get_secret("OPENAI_API_KEY")

In [4]:
!pip install transformers==4.40.1 bitsandbytes==0.43.1 accelerate==0.29.3 datasets==2.19.0 tiktoken==0.6.0 -qqq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 3.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 65.8 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 13.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 62.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.0/172.0 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 79.1 MB/s eta 0:00:00:00:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.12.0 requires pyarrow<19.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 19.0.0 which is incompatible.
gcsfs 2024.10.0 r

## 데이터셋 관련
- 실습 데이터 (GPT로 생성) : https://huggingface.co/datasets/shangrilar/ko_text2sql/viewer/origin
- 게임 DB에 맞게 형성

### 데이터 로딩
- 영어 text2sql : https://huggingface.co/datasets/Salesforce/wikisql
- 한글 text2sql : https://www.aihub.or.kr/aihubdata/data/view.do?currMenu=&topMenu=&aihubDataSe=data&dataSetSn=71351


In [7]:
from datasets import load_dataset
text2sql_dataset = load_dataset("shangrilar/ko_text2sql",'origin')
text2sql_dataset

Generating train split:   0%|          | 0/38246 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/112 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['db_id', 'context', 'question', 'answer'],
        num_rows: 38246
    })
    test: Dataset({
        features: ['db_id', 'context', 'question', 'answer'],
        num_rows: 112
    })
})

In [13]:
# huggingface dataset to pandas dataframe
text2sql_train_df = text2sql_dataset['train'].to_pandas()
text2sql_test_df = text2sql_dataset['test'].to_pandas()
text2sql_train_df.head()
text2sql_test_df.head()

,db_id,context,question,answer
0,1,CREATE TABLE quests (\n quest_id INT PRIMARY ...,각 보상 아이템별로 보상 경험치의 합을 구해줘,"SELECT reward_items, SUM(reward_experience) AS..."
1,1,CREATE TABLE players (\n player_id INT PRIMAR...,사용자 이름에 'admin'이 포함되어 있는 계정의 수를 알려주세요.,SELECT COUNT(*) FROM players WHERE username LI...
2,1,CREATE TABLE quests (\n quest_id INT PRIMARY ...,퀘스트 진행 상황이 100%인 퀘스트의 이름과 보상 경험치는 얼마인가요?,"SELECT q.name, q.reward_experience FROM quests..."
3,1,CREATE TABLE characters (\n character_id INT ...,경험이 5000000 이상이거나 직업이 전사인 캐릭터들의 이름은 무엇인가,SELECT name FROM characters WHERE experience >...
4,1,CREATE TABLE characters (\n character_id INT ...,레벨이 20 이상인 플레이어의 캐릭터 이름과 해당 캐릭터의 스킬 이름을 알아보세요.,"SELECT C.name, ST.skill_name FROM characters A..."


In [15]:
text2sql_train_df = text2sql_train_df.dropna()
text2sql_train_df = text2sql_train_df.query('db_id != 1') # 정상 query 만 사용
text2sql_train_df.head()

,db_id,context,question,answer
2388,2,"CREATE TABLE messages (\n ""message_id"" SERI...",messages 테이블에서 모든 데이터를 조회해줘,SELECT * FROM messages;
2389,2,"CREATE TABLE messages (\n ""message_id"" SERI...",보낸 사람의 ID가 15보다 큰 메시지들의 수를 알려줘,SELECT COUNT(*) FROM messages WHERE sender_id ...
2390,2,"CREATE TABLE messages (\n ""message_id"" SERI...",읽지 않은 메시지들은 몇 개인가요?,SELECT COUNT(*) FROM messages WHERE read = FALSE;
2391,2,"CREATE TABLE messages (\n ""message_id"" SERI...",대화 ID가 42인 대화에서 보낸 메시지들은 몇 개인가요?,SELECT COUNT(*) FROM messages WHERE conversati...
2392,2,"CREATE TABLE messages (\n ""message_id"" SERI...",보낸 시간이 2023-01-01 이후인 메시지의 수를 구해줘,SELECT COUNT(*) FROM messages WHERE timestamp ...


In [18]:
def make_prompt(ddl, question, query=''):
    prompt = f"""당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question을 해결할 수 있는 SQL 쿼리를 생성하세요.
### DDL:
{ddl}
### Question:
{question}
### SQL:
{query}"""
    return prompt

In [19]:
make_prompt(text2sql_train_df['context'],text2sql_train_df['question'],text2sql_train_df['answer'])

'당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question을 해결할 수 있는 SQL 쿼리를 생성하세요.\n### DDL:\n2388     CREATE TABLE messages (\\n    "message_id" SERI...\n2389     CREATE TABLE messages (\\n    "message_id" SERI...\n2390     CREATE TABLE messages (\\n    "message_id" SERI...\n2391     CREATE TABLE messages (\\n    "message_id" SERI...\n2392     CREATE TABLE messages (\\n    "message_id" SERI...\n                               ...                        \n38241    CREATE TABLE users (\\n    "user_id" SERIAL PRI...\n38242    CREATE TABLE users (\\n    "user_id" SERIAL PRI...\n38243    CREATE TABLE users (\\n    "user_id" SERIAL PRI...\n38244    CREATE TABLE users (\\n    "user_id" SERIAL PRI...\n38245    CREATE TABLE users (\\n    "user_id" SERIAL PRI...\nName: context, Length: 33876, dtype: object\n### Question:\n2388                           messages 테이블에서 모든 데이터를 조회해줘\n2389                        보낸 사람의 ID가 15보다 큰 메시지들의 수를 알려줘\n2390                                   읽지 않은 메시지들은 몇 개인가요?\n2391  

In [21]:
# 필요한 컬럼 추출해 프롬프트 명령어 만들기
from tqdm import tqdm

for idx, row in tqdm(text2sql_train_df.iterrows(),
                     total = len(text2sql_train_df), 
                     desc='Generating prompt'): # DataFrame row return
    prompt_command = make_prompt(row['context'], 
                         row['question'], 
                         row['answer']
                        )
    text2sql_train_df.loc[idx, 'prompt'] = prompt_command
    pass
text2sql_train_df.head()

Generating prompt: 100%|██████████| 33876/33876 [00:07<00:00, 4575.38it/s]


,db_id,context,question,answer,prompt
2388,2,"CREATE TABLE messages (\n ""message_id"" SERI...",messages 테이블에서 모든 데이터를 조회해줘,SELECT * FROM messages;,당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question...
2389,2,"CREATE TABLE messages (\n ""message_id"" SERI...",보낸 사람의 ID가 15보다 큰 메시지들의 수를 알려줘,SELECT COUNT(*) FROM messages WHERE sender_id ...,당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question...
2390,2,"CREATE TABLE messages (\n ""message_id"" SERI...",읽지 않은 메시지들은 몇 개인가요?,SELECT COUNT(*) FROM messages WHERE read = FALSE;,당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question...
2391,2,"CREATE TABLE messages (\n ""message_id"" SERI...",대화 ID가 42인 대화에서 보낸 메시지들은 몇 개인가요?,SELECT COUNT(*) FROM messages WHERE conversati...,당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question...
2392,2,"CREATE TABLE messages (\n ""message_id"" SERI...",보낸 시간이 2023-01-01 이후인 메시지의 수를 구해줘,SELECT COUNT(*) FROM messages WHERE timestamp ...,당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question...


In [23]:
!mkdir ./datasets

In [24]:
# 재사용 위한 저장
dataset_path = f'datasets'
text2sql_train_df.to_csv(f'{dataset_path}/train.csv')

In [26]:
for idx, row in tqdm(text2sql_test_df.iterrows(),
                     total = len(text2sql_test_df), 
                     desc='Generating prompt'): # DataFrame row return
    prompt_command = make_prompt(row['context'], 
                         row['question'], 
                         ''
                        )
    text2sql_test_df.loc[idx, 'prompt'] = prompt_command
    pass
text2sql_test_df.head()

Generating prompt: 100%|██████████| 112/112 [00:00<00:00, 2057.04it/s]


,db_id,context,question,answer,prompt
0,1,CREATE TABLE quests (\n quest_id INT PRIMARY ...,각 보상 아이템별로 보상 경험치의 합을 구해줘,"SELECT reward_items, SUM(reward_experience) AS...",당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question...
1,1,CREATE TABLE players (\n player_id INT PRIMAR...,사용자 이름에 'admin'이 포함되어 있는 계정의 수를 알려주세요.,SELECT COUNT(*) FROM players WHERE username LI...,당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question...
2,1,CREATE TABLE quests (\n quest_id INT PRIMARY ...,퀘스트 진행 상황이 100%인 퀘스트의 이름과 보상 경험치는 얼마인가요?,"SELECT q.name, q.reward_experience FROM quests...",당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question...
3,1,CREATE TABLE characters (\n character_id INT ...,경험이 5000000 이상이거나 직업이 전사인 캐릭터들의 이름은 무엇인가,SELECT name FROM characters WHERE experience >...,당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question...
4,1,CREATE TABLE characters (\n character_id INT ...,레벨이 20 이상인 플레이어의 캐릭터 이름과 해당 캐릭터의 스킬 이름을 알아보세요.,"SELECT C.name, ST.skill_name FROM characters A...",당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question...


In [28]:
text2sql_test_df['prompt'][0]

'당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question을 해결할 수 있는 SQL 쿼리를 생성하세요.\n### DDL:\nCREATE TABLE quests (\n  quest_id INT PRIMARY KEY AUTO_INCREMENT,\n  name VARCHAR(255) NOT NULL,\n  description TEXT,\n  reward_experience INT NOT NULL,\n  reward_items VARCHAR(255)\n);\n### Question:\n각 보상 아이템별로 보상 경험치의 합을 구해줘\n### SQL:\n'

In [29]:
text2sql_test_df.to_csv(f'{dataset_path}/test.csv')

## linux 명령어 알기

In [33]:
!ls -la ../

total 20
drwxr-xr-x 5 root root 4096 Feb 10 02:23 .
drwxr-xr-x 1 root root 4096 Feb 10 02:23 ..
drwxr-xr-x 2 root root 4096 Feb 10 02:23 input
drwxr-xr-x 3 root root 4096 Feb 10 02:23 lib
drwxr-xr-x 4 root root 4096 Feb 10 03:15 working


In [34]:
!pwd

/kaggle/working


In [35]:
!zip -r generativeLLM_HF_dataset.zip *

  adding: datasets/ (stored 0%)
  adding: datasets/train.csv (deflated 95%)
  adding: datasets/test.csv (deflated 92%)


In [36]:
!ls -al

total 2380
drwxr-xr-x 4 root root    4096 Feb 10 03:26 .
drwxr-xr-x 5 root root    4096 Feb 10 02:23 ..
drwxr-xr-x 2 root root    4096 Feb 10 03:20 datasets
-rw-r--r-- 1 root root 2420718 Feb 10 03:26 generativeLLM_HF_dataset.zip
drwxr-xr-x 2 root root    4096 Feb 10 02:24 .virtual_documents
